In [1]:
import pandas as pd

c2n = pd.read_excel('CSA-to-NSA-2010.xlsx')
hinc = pd.read_excel('Vital Signs Indicator Median Household Income.xlsx')
z2c = pd.read_excel('Zip-to-CSA-2010.xls')
df = pd.read_csv('MDCOVID19_MASTER_ZIP_CODE_CASES.csv')
census = pd.read_csv('Census_Demographics_2010-2012.csv')

In [2]:
#pipeline raw data -> cleaned rows:dates, columns:zips------------------------
zipsList = [21227,21207,21230,21251,21229,21237,21287,21231,21226,21206,21222,21225,21211,21208,\
            21205,21218,21234,21202,21201,21213,21210,21209,21216,21236,21217,21224,21215,21212,\
                21223,21214,21239,21228]

df = df[df['ZIP_CODE'].isin(zipsList)]

df = df.fillna(0)
df = df.drop('OBJECTID', axis=1)

df = df.transpose()

df.columns = df.iloc[0]
df = df.drop('ZIP_CODE')
df = df.reset_index()
df = df.rename(columns={'index':'Date'})

df['Date'] = df['Date'].map(lambda x: x.lstrip('F').lstrip('t').lstrip('o').lstrip('t')\
                            .lstrip('a').lstrip('l'))

dates = pd.DataFrame()
dates = df['Date'].str.split(pat='_', expand=True)
dates[0] = dates[0].map(lambda x: x.lstrip('0'))
dates['fdate'] = dates[2].str.cat(dates[0], sep='-').str.cat(dates[1],sep='-')

dates = dates['fdate']

df = df.drop('Date',axis=1)

df.insert(0,'Date',dates)

df['Date'] = pd.to_datetime(df['Date'])

print(df.head())
df.to_csv('COVID_Cleaned.csv')

ZIP_CODE       Date  21201.0  21202.0  21205.0  21206.0  21207.0  21208.0  \
0        2020-04-11     17.0     32.0     18.0     89.0     71.0     89.0   
1        2020-04-12     18.0     35.0     19.0     96.0     76.0    106.0   
2        2020-04-13     19.0     37.0     20.0     98.0     79.0    108.0   
3        2020-04-14     21.0     37.0     21.0    101.0     87.0    114.0   
4        2020-04-15     21.0     37.0     25.0    110.0     92.0    119.0   

ZIP_CODE  21209.0  21210.0  21211.0  ...  21228.0  21229.0  21230.0  21231.0  \
0            60.0      0.0     14.0  ...     51.0     62.0     39.0     29.0   
1            64.0      8.0     14.0  ...     52.0     65.0     41.0     30.0   
2            64.0      8.0     14.0  ...     53.0     71.0     43.0     30.0   
3            66.0      8.0     16.0  ...     55.0     76.0     43.0     30.0   
4            69.0      8.0     16.0  ...     63.0     88.0     45.0     31.0   

ZIP_CODE  21234.0  21236.0  21237.0  21239.0  21251.0  2

In [3]:
#pipeline rows:dates, columns:zips -> rows:zips, columns:zips-----------------
df = df.transpose()
#df.to_csv('test.csv')
#df = df.drop('Unnamed: 0')
df = df.reset_index()

df.columns = df.iloc[0]

df = df.rename(columns={'Date':'Zip Code'})
df1 = df.drop([0,0])

print(df1.head())
df1.to_csv('COVID_Cleaned_Transposed.csv')

0 Zip Code 2020-04-11 00:00:00 2020-04-12 00:00:00 2020-04-13 00:00:00  \
1    21201                  17                  18                  19   
2    21202                  32                  35                  37   
3    21205                  18                  19                  20   
4    21206                  89                  96                  98   
5    21207                  71                  76                  79   

0 2020-04-14 00:00:00 2020-04-15 00:00:00 2020-04-16 00:00:00  \
1                  21                  21                  22   
2                  37                  37                  38   
3                  21                  25                  27   
4                 101                 110                 114   
5                  87                  92                  96   

0 2020-04-17 00:00:00 2020-04-18 00:00:00 2020-04-19 00:00:00  ...  \
1                  25                  26                  27  ...   
2                  40   

In [4]:
#pipeline zips, CSAs, NSAs, MHINC, CSA pop, total COVID cases -> one file-----
zipsLatest = pd.DataFrame()

zipsLatest['Zip'] = df1['Zip Code'].astype(int)
zipsLatest['TotalCOVIDCases'] = df1.iloc[:,-1]

census = census[['CSA2010','tpop10']]
census = census.rename(columns={'CSA2010':'Community','tpop10':'Pop2010'})
#rename columns appropriately
z2c = z2c.rename(columns={'Zip2010':'Zip','CSA2010':'CSA'})
c2n = c2n.rename(columns={'CSA2010':'Community'})

#merge, rename Household income, CSA population, and CSAs-to-Neighborhoods
df = pd.merge_ordered(hinc, c2n, fill_method='ffill')
df = pd.merge_ordered(census,df,fill_method='ffill')
df = df.rename(columns={'2018 Data':'MHINC','NSA2010':'Neigh','Community':'CSA'})
df = df[['Neigh','CSA','Pop2010','MHINC']]

df = df.drop([0])
df.to_csv('test.csv')
#merge with Zip Codes
df = pd.merge_ordered(z2c,df,fill_method='ffill')

#drop commas from household income numbers, convert to float
df['MHINC'] = df['MHINC'].replace(',','',regex=True)
df['MHINC'] = df['MHINC'].astype(float)

#convert zips to ints
df['Zip'] = df['Zip'].astype(int)

#merge with latest covid cases
df = pd.merge_ordered(zipsLatest,df,fill_method='ffill')
df = df[['Neigh','CSA','Pop2010','Zip','MHINC','TotalCOVIDCases']]

#export as CSV
df.to_csv('MASTER_MERGED.csv',index=False)
print(df.head())

                    Neigh                        CSA  Pop2010    Zip  \
0                Downtown        Downtown/Seton Hill     6446  21201   
1              Seton Hill        Downtown/Seton Hill     6446  21201   
2  University of Maryland        Downtown/Seton Hill     6446  21201   
3           Downtown West  Inner Harbor/Federal Hill    12855  21201   
4            Federal Hill  Inner Harbor/Federal Hill    12855  21201   

      MHINC TotalCOVIDCases  
0   57516.7             447  
1   57516.7             447  
2   57516.7             447  
3  100932.2             447  
4  100932.2             447  


In [5]:
#pipeline COVID_Cleaned_Transposed -> Daily Increase--------------------------
cc = pd.read_csv('COVID_Cleaned.csv')

cc = cc.transpose()
cc = cc.drop('Unnamed: 0')
cc = cc.reset_index()

cc.columns = cc.iloc[0]

cc = cc.rename(columns={'Date':'Zip Code'})
cc = cc.drop([0,0]) 

cc1 = cc.loc[:,'2020-04-11':]\
        .diff(axis=1)\
        .drop(columns='2020-04-11')\
  
dates = cc['Zip Code']
cc1.insert(0,'Zip Code',dates)    

cc.to_csv('Bmore_COVID_NewCasesByDate.csv')
print(cc.head())

0 Zip Code 2020-04-11 2020-04-12 2020-04-13 2020-04-14 2020-04-15 2020-04-16  \
1  21201.0         17         18         19         21         21         22   
2  21202.0         32         35         37         37         37         38   
3  21205.0         18         19         20         21         25         27   
4  21206.0         89         96         98        101        110        114   
5  21207.0         71         76         79         87         92         96   

0 2020-04-17 2020-04-18 2020-04-19  ... 2020-11-08 2020-11-09 2020-11-10  \
1         25         26         27  ...        388        395        401   
2         40         40         49  ...        753        757        765   
3         28         29         34  ...        616        617        624   
4        121        124        134  ...       1796       1812       1832   
5        107        110        116  ...       1570       1584       1602   

0 2020-11-11 2020-11-12 2020-11-13 2020-11-14 2020-11-15 2020-

In [6]:
#pipeline COVID_Cleaned_Transposed -> Daily Increase--------------------------
cc = pd.read_csv('COVID_Cleaned.csv')

cc = cc.transpose()
cc = cc.drop('Unnamed: 0')
cc = cc.reset_index()

cc.columns = cc.iloc[0]

cc = cc.rename(columns={'Date':'Zip Code'})
cc = cc.drop([0,0]) 

cc1 = cc.loc[:,'2020-04-11':]\
        .diff(axis=1)\
        .drop(columns='2020-04-11')\
  
dates = cc['Zip Code']
cc1.insert(0,'Zip Code',dates)    

cc.to_csv('Bmore_COVID_NewCasesByDate.csv')
print(cc.head())

#pipeline -> Mean, median new cases, total cases
latestCases = pd.DataFrame()

latestCases['Zip Code'] = df1['Zip Code']
latestCases['Total Cases'] = df1.iloc[:,-1]

latestCases['Zip Code'] = latestCases['Zip Code'].astype(int)
latestCases['Total Cases'] = latestCases['Total Cases'].astype(int)

meanList = []
medianList = []

#loop through rows to calculate mean/median new cases per day per zip code
for x in range(0,len(cc.index)):
    #add values to list
    meanList.append(cc.iloc[x,2:].mean())
    medianList.append(cc.iloc[x,2:].median())

#add to latest cases
latestCases.insert(2,'meanNewCasesPerDay',meanList)
latestCases.insert(3,'medianNewCasesPerDay',medianList)

print(latestCases.head())
latestCases.to_csv('TotalMeanMedianCOVIDbyZip.csv')

0 Zip Code 2020-04-11 2020-04-12 2020-04-13 2020-04-14 2020-04-15 2020-04-16  \
1  21201.0         17         18         19         21         21         22   
2  21202.0         32         35         37         37         37         38   
3  21205.0         18         19         20         21         25         27   
4  21206.0         89         96         98        101        110        114   
5  21207.0         71         76         79         87         92         96   

0 2020-04-17 2020-04-18 2020-04-19  ... 2020-11-08 2020-11-09 2020-11-10  \
1         25         26         27  ...        388        395        401   
2         40         40         49  ...        753        757        765   
3         28         29         34  ...        616        617        624   
4        121        124        134  ...       1796       1812       1832   
5        107        110        116  ...       1570       1584       1602   

0 2020-11-11 2020-11-12 2020-11-13 2020-11-14 2020-11-15 2020-